In [1]:
%reset
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
from xgcm import Grid

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## We read in data that has been high-pass filtered using GCM-filters.

In [15]:
#1/4 degree resolution, surface diagnostics are for uppermost 5 meters.
%cd /glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/nw2_0.25deg_N15_baseline_hmix5/
%ls
fs = xr.open_dataset('static.nc', decode_times=False)
test= xr.open_dataset('MOM_IC.nc', decode_times=False)
os = xr.open_mfdataset(['sub01/ocean.stats.nc','sub02/ocean.stats.nc'], decode_times=False)
av = xr.open_dataset('/glade/scratch/noraloose/filtered_data/neverworld2-paper/baselines/nw2_0.25deg_N15_baseline_hmix5/av_f_high_Gaussian.nc', decode_times=False)

# #1/8 degree resolution, surface diagnostics are for uppermost 5 meters.
# %cd /glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/nw2_0.125deg_N15_baseline_hmix5/
# fs = xr.open_dataset('static.nc', decode_times=False)
# os = xr.open_mfdataset(['sub01/ocean.stats.nc','sub02/ocean.stats.nc'], decode_times=False)
# av = xr.open_dataset('/glade/scratch/noraloose/filtered_data/neverworld2-paper/baselines/nw2_0.125deg_N15_baseline_hmix5/av_f_high_Gaussian.nc', decode_times=False)

# # #1/16 degree resolution, surface diagnostics are for uppermost 5 meters.
# %cd /glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/nw2_0.0625deg_N15_baseline_hmix5/
# fs = xr.open_dataset('static.nc', decode_times=False)
# os = xr.open_mfdataset(['sub01/ocean.stats.nc','sub02/ocean.stats.nc','sub03/ocean.stats.nc','sub04/ocean.stats.nc',
#                          'sub05/ocean.stats.nc'], decode_times=False)
# av = xr.open_dataset('/glade/scratch/noraloose/filtered_data/neverworld2-paper/baselines/nw2_0.0625deg_N15_baseline_hmix5/av_f_high_Gaussian.nc', decode_times=False)

# #1/32 degree resolution, surface diagnostics are for uppermost 5 meters.
# %cd /glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/nw2_0.03125deg_N15_baseline_hmix5
# fs = xr.open_dataset('static.nc', decode_times=False)
# os = xr.open_mfdataset(['sub01/ocean.stats.nc','sub02/ocean.stats.nc','sub03/ocean.stats.nc','sub04/ocean.stats.nc',
#                         'sub05/ocean.stats.nc','sub06/ocean.stats.nc','sub07/ocean.stats.nc','sub08/ocean.stats.nc',
#                         'sub09/ocean.stats.nc','sub10/ocean.stats.nc','sub11/ocean.stats.nc','sub12/ocean.stats.nc',
#                         'sub13/ocean.stats.nc','sub14/ocean.stats.nc','sub15/ocean.stats.nc','sub16/ocean.stats.nc',
#                         'sub17/ocean.stats.nc','sub18/ocean.stats.nc','sub19/ocean.stats.nc','sub20/ocean.stats.nc'], decode_times=False)
# av = xr.open_dataset('/glade/scratch/noraloose/filtered_data/neverworld2-paper/baselines/nw2_0.03125deg_N15_baseline_hmix5/av_f_high_Gaussian.nc', decode_times=False)

%cd /glade/p/univ/unyu0004/eyankovsky/NeverWorld_analysis/


/glade/campaign/univ/unyu0004/NeverWorld2/nw2_0.25deg_N15_baseline_hmix5
available_diags.000000  longmean_00010050.nc         MOM_parameter_doc.layout
averages_00030002.nc    longmean_00012050.nc         MOM_parameter_doc.short
averages_00030502.nc    longmean_00014050.nc         ocean_geometry.nc
averages_00031002.nc    longmean_00015050.nc         ocean_stats.nc
averages_00031502.nc    longmean_00016050.nc         RESTART/
CPU_stats               longmean_00018050.nc         snapshots_00030005.nc
diag_table@             longmean_00020050.nc         snapshots_00030505.nc
diag_table.final        longmean_00022050.nc         snapshots_00031005.nc
diag_table.init         longmean_00024050.nc         snapshots_00031505.nc
exitcode                longmean_00026050.nc         static.nc
INPUT/                  longmean_00028050.nc         sub01/
input.nml               longmean_00030050.nc         sub02/
logfile.000000.out      MOM_IC.nc                    sub03/
longmean_00000050.nc    MOM_

In [16]:
depth = fs['depth_ocean']
area_t=fs['area_t']


interface_ref=os['H0'].mean('Time').load(); interface_ref=interface_ref[:-1]
Layer = (os['Layer']);
drho=xr.zeros_like(interface_ref)
drho[0]=Layer[0].values; drho[1:]=np.diff(Layer)
print(interface_ref); #drho contains the density differences across each interface. 15 entries, includes surface but not bottom.


<xarray.DataArray 'H0' (Interface: 15)>
array([8.31083442e-11, 2.50000000e+01, 7.50000000e+01, 1.75000000e+02,
       3.00000000e+02, 4.50000000e+02, 6.25000000e+02, 8.25000000e+02,
       1.05000000e+03, 1.30000000e+03, 1.60000000e+03, 1.95000000e+03,
       2.35000000e+03, 2.85000000e+03, 3.40000000e+03])
Coordinates:
  * Interface  (Interface) float64 1.022e+03 1.023e+03 ... 1.028e+03 1.028e+03


In [17]:
h = (av['h'][:,:,:,:])
correction = (interface_ref-depth); #if >0 this means h_ref is deeper than topography, we need correction!
correction = correction.where(correction>0.0).fillna(0) #if correction <0 that means depth is larger than h_ref, we don't need correction!

interface = xr.zeros_like(h)
for t in range (0,100):
    for i in range(0,len(Layer)):
        interface[t,i,:,:]=h[t,i:,:,:].sum(axis=0)-depth
        #interface is negative down
    #print(t)
interface_ref_HIC = interface_ref- correction
interface_ref_HIC=interface_ref_HIC.rename({'Interface': 'zl'}) 


drho=drho.rename({'Interface': 'zl'})
interface_ref_HIC.coords['zl'] = h['zl']; drho.coords['zl'] = h['zl']


In [18]:
eta2 = (interface+interface_ref_HIC)**2
APE_net=(0.5*10*drho*eta2*area_t).sum('zl') #Gives units of Joules just like the os file.
print('done')

done


## The next line prints out the APE from our offline computation as well as the APE based on the model diagnostic (units Joules)

In [19]:
print(np.nansum(APE_net[-100:,:,:].mean('time'))) #Computed APE
print(np.nansum(os.APE[-100:,:].mean('Time')))    #Model diagnostic APE


1.832003541669497e+20
1.0640613739735328e+20


In [20]:
u = (av['u'][:,:,:,:])
v = (av['v'][:,:,:,:])

KE=(0.5*(v**2+u**2)*h*area_t).sum('zl')

## The next line prints out the KE from our offline computation as well as the KE based on the model diagnostic (units Joules)


In [21]:
print(np.nansum(KE[-100:,:,:].mean('time'))*1000) #Computed EE
print(np.nansum(os.KE[-100:,:].mean('Time')))    #Model diagnostic KE


5.3135708159277114e+17
5.95674968759358e+17
